# Imports

In [2]:
import sys
import os

current_dir = os.path.abspath(os.getcwd())
parent_dir = os.path.dirname(current_dir)
sys.path.append(os.path.join(parent_dir, 'scripts'))

import data_acquisition
import Dataframe


# Collecting Data

In [3]:
data_acquisition.collect_data()

fetching data for 2016_2017 ..


Fetching game data for 2016-2017 Season:   0%|          | 0/1335 [00:00<?, ?it/s]

Fetching game data for 2016-2017 Season: 100%|██████████| 1335/1335 [01:58<00:00, 11.29it/s]


saving data for 2016_2017 ..
The folder '../data/raw/2017_2018' already exists.
The folder '../data/raw/2018_2019' already exists.
The folder '../data/raw/2019_2020' already exists.
The folder '../data/raw/2020_2021' already exists.


# Cleaning Data

In [3]:
base_path = '../data/raw/'
df = Dataframe.load_all_seasons(base_path)
transformed_df = Dataframe.transformEventData(df)

Seasons: 100%|██████████| 5/5 [02:25<00:00, 29.08s/it]
